# Homework 8: Large Language Models

An PDF overview of the homework is [here](https://www.cs.jhu.edu/~jason/465/hw-llm/).

It mentions: "We'll send hand-in instructions soon.  Probably we will ask you to submit a version
of the main notebook, with your answers added and extraneous materials deleted. We may also
ask for a summary."

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
This symbol marks a question or exercise that you will be expected to hand in.

# Getting started

## Activate `conda` environment

When executing cells in this notebook, you will need to connect to an `nlp-class` kernel, which is a Python process running in that environment.  This is the notebook equivalent of the terminal command `conda activate nlp-class`.  

If you need to create or update that environment, first download the [nlp-class.yml](http://cs.jhu.edu/~jason/465/hw-llm/nlp-class.yml) file, and execute
```
conda env update --file nlp-class.yml --prune
```

## Fetch code and data files for this homework

All of the files you need are in the directory <https://www.cs.jhu.edu/~jason/465/hw-llm/>.  To get a local copy of that directory, including this notebook, you can download and unpack [HW-LLM.zip](https://www.cs.jhu.edu/~jason/465/hw-llm/HW-LLM.zip).  Then open this notebook.

Note that the other files must be in the *same directory* as this notebook.  Otherwise, a command like `import tracking` won't be able to find the tracking module, `tracking.py`.



In [16]:
# Check that the current directory does contain the files.
!ls -lR *.py data

-rw-r--r--@ 1 emilyguan  staff  20118 Dec  7 14:03 agents.py
-rw-r--r--@ 1 emilyguan  staff   5391 Dec  7 17:22 argubots.py
-rw-r--r--@ 1 emilyguan  staff   2838 Dec  7 14:03 characters.py
-rw-r--r--@ 1 emilyguan  staff   2641 Dec  7 14:03 dialogue.py
-rw-r--r--@ 1 emilyguan  staff  14250 Dec  7 14:03 evaluate.py
-rw-r--r--@ 1 emilyguan  staff  10426 Dec  7 14:03 kialo.py
-rw-r--r--@ 1 emilyguan  staff   1347 Dec  7 14:03 logging_cm.py
-rw-r--r--@ 1 emilyguan  staff   1503 Dec  7 14:03 simulate.py
-rw-r--r--@ 1 emilyguan  staff   7157 Dec  7 14:03 tracking.py

data:
total 4512
-rw-rw-r--@ 1 emilyguan  staff     407 Dec  7 14:20 LICENSE
-rw-rw-r--@ 1 emilyguan  staff  613106 Dec  7 14:20 all-humans-should-be-vegan-2762.txt
-rw-rw-r--@ 1 emilyguan  staff   81917 Dec  7 14:20 have-authoritarian-governments-handled-covid-19-better-than-others-54145.txt
-rw-rw-r--@ 1 emilyguan  staff   52771 Dec  7 14:20 is-biden-an-incompetent-president-44217.txt
-rw-rw-r--@ 1 emilyguan  staff  153551 Dec 


The `autoreload` feature of Jupyter ensures that if an imported module (.py file) changes, the notebook will automatically import the new version.  
(However, objects that were defined with the old version of the class won't change.)

In [17]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create an OpenAI client

An OpenAI API key will be sent to you.  (Or are you not in the class? Then you can make your own API key by [signing up for an OpenAI platform account](https://platform.openai.com/signup) and putting some money on it.  This assignment should cost only about $1 US.)

Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]    # do not include the brackets here
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.



Now you can execute the following to get an OpenAI client object.

In [18]:
from tracking import new_default_client, read_usage
client = new_default_client() 

That fetches your API key and calls `openai.OpenAI()` to make a new **client** object, whose job is to talk to the OpenAI **server** over HTTP.  (The `OpenAI` constructor has some optional arguments that configure these HTTP messages.
However, the defaults should work fine for you.)

That command also saved the new client in `tracking.default_client`, which is the client that the starter code will use by default whenever it needs to talk to the OpenAI server.  Thus, you should **rerun the above cell** to get a new client if you change the `default_model` in `tracking.py`, or if your API key in  `.env` ever changes, or its associated organization ever changes.

## Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.  
You will have to specify which OpenAI model to use.
Documentation of the methods is [here](https://pypi.org/project/openai/) if you are curious.

### Continue a textual prompt

This is what language models excel at.  In principle you should do it by calling [`client.completions.create`](https://platform.openai.com/docs/api-reference/completions/create?lang=python).  However, OpenAI has [retired](https://openai.com/blog/gpt-4-api-general-availability) most of the models that support that API (keeping only `gpt-3.5-turbo-instruct`).  So we'll use the more modern API, [`client.chat.completions.create`](https://platform.openai.com/docs/api-reference/chat/create?lang=python).

In [19]:
import rich   # prettyprinting

response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": "Q: Name the planets in the solar system?\nA: "}], 
                                          model="gpt-3.5-turbo",       # which model to use
                                          temperature=1,               # get a little variety
                                          max_tokens=64,               # limit on length of result
                                          logprobs=True,
                                          top_logprobs=5
                                          # stop=["Q:", "\n"],         # treat these as EOS symbols; useful for some models
                                         )           
rich.print(response)                              # the full object that was sent back from the server
rich.print(response.choices)                      # just the list of 1 answer (the default, but calling with n=5 would give 5 answers) 
rich.print(response.choices[0].message.content)   # extract the good stuff from that 1 answer

ChatCompletion(
    id='chatcmpl-Cm7ZxcnS2kcU4dDXQhDMV988IMSqh',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=ChoiceLogprobs(
                content=[
                    ChatCompletionTokenLogprob(
                        token='1',
                        bytes=[49],
                        logprob=-0.31794053,
                        top_logprobs=[
                            TopLogprob(token='1', bytes=[49], logprob=-0.31794053),
                            TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.558131),
                            TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1224926),
                            TopLogprob(token='-', bytes=[45], logprob=-4.436551),
                            TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.858014)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='.',
                        bytes=[46],
                        logprob=-0.0011530785,
                        top_logprobs=[
                            TopLogprob(token='.', bytes=[46], logprob=-0.0011530785),
                            TopLogprob(token=')', bytes=[41], logprob=-6.772958),
                            TopLogprob(token='-', bytes=[45], logprob=-12.652219),
                            TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242631),
                            TopLogprob(token=':', bytes=[58], logprob=-14.006652)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token=' Mercury',
                        bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                        logprob=-1.6286107e-05,
                        top_logprobs=[
                            TopLogprob(
                                token=' Mercury',
                                bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                                logprob=-1.6286107e-05
                            ),
                            TopLogprob(token=' ', bytes=[32], logprob=-11.215232),
                            TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.03682),
                            TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.565732),
                            TopLogprob(token='  ', bytes=[32, 32], logprob=-15.394105)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='\n',
                        bytes=[10],
                        logprob=-0.02446258,
                        top_logprobs=[
                            TopLogprob(token='\n', bytes=[10], logprob=-0.02446258),
                            TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.746848),
                            TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.549508),
                            TopLogprob(token='\n\n', bytes=[10, 10], logprob=-10.689649),
                            TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.702931)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='2',
                        bytes=[50],
                        logprob=-4.00813e-06,
                        top_logprobs=[
                            TopLogprob(token='2', bytes=[50], logprob=-4.00813e-06),
                            TopLogprob(token=' ', bytes=[32], logprob=-13.296427),
                            TopLogprob(token='1', bytes=[49], logprob=-13.68288),
                            TopLogprob(token='3', bytes=[51], logprob=-14.106123),
                            TopLogprob(token='  ', bytes=[32, 32], logprob=-15.547543)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='.',
         

[
    Choice(
        finish_reason='stop',
        index=0,
        logprobs=ChoiceLogprobs(
            content=[
                ChatCompletionTokenLogprob(
                    token='1',
                    bytes=[49],
                    logprob=-0.31794053,
                    top_logprobs=[
                        TopLogprob(token='1', bytes=[49], logprob=-0.31794053),
                        TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.558131),
                        TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1224926),
                        TopLogprob(token='-', bytes=[45], logprob=-4.436551),
                        TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.858014)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-0.0011530785,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-0.0011530785),
                        TopLogprob(token=')', bytes=[41], logprob=-6.772958),
                        TopLogprob(token='-', bytes=[45], logprob=-12.652219),
                        TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242631),
                        TopLogprob(token=':', bytes=[58], logprob=-14.006652)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token=' Mercury',
                    bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                    logprob=-1.6286107e-05,
                    top_logprobs=[
                        TopLogprob(
                            token=' Mercury',
                            bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                            logprob=-1.6286107e-05
                        ),
                        TopLogprob(token=' ', bytes=[32], logprob=-11.215232),
                        TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.03682),
                        TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.565732),
                        TopLogprob(token='  ', bytes=[32, 32], logprob=-15.394105)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='\n',
                    bytes=[10],
                    logprob=-0.02446258,
                    top_logprobs=[
                        TopLogprob(token='\n', bytes=[10], logprob=-0.02446258),
                        TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.746848),
                        TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.549508),
                        TopLogprob(token='\n\n', bytes=[10, 10], logprob=-10.689649),
                        TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.702931)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='2',
                    bytes=[50],
                    logprob=-4.00813e-06,
                    top_logprobs=[
                        TopLogprob(token='2', bytes=[50], logprob=-4.00813e-06),
                        TopLogprob(token=' ', bytes=[32], logprob=-13.296427),
                        TopLogprob(token='1', bytes=[49], logprob=-13.68288),
                        TopLogprob(token='3', bytes=[51], logprob=-14.106123),
                        TopLogprob(token='  ', bytes=[32, 32], logprob=-15.547543)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-6.749814e-06,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-6.749814e-06),
                        TopLogprob(token=' Venus', bytes=[32, 86, 101, 110, 117, 115], logprob=-12.418757),
                        TopLogprob(token=' .', bytes=[32, 46], logprob

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try running the cell above a few times. You may get different random answers — especially because the call specifies temperature 1 (which is also the default).  Are the answers all equally good?

A: All answers were equally correct, they only differed with respect to formatting. One could argue that this means they were equally good. Some responses were numbered, while others were lists. Some of them contained longer responses that repeated the question back at the user.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try adding the arguments `logprobs=True, top_logprobs=5` to the above API call (see [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat-create-logprobs)).  For each generated token, the response will now include its log-probability, and also the log-probabilities of the 5 most probable tokens, given the left context so far.  Again, run the cell a few times.  What do you observe?


A: Although the final response output is the same as before, we are now able to see the log-probability of top tokens. Interestingly, the system oscillates between choosing the whole word/name of the planet and using  subotokens. For example, the model for the most part will choose Uranus over Uran, but it will occasionally choose Uran->us


It might be handy to package up what we just did.
The `complete` function below is a convenient way of experimenting with completing text.
It is illustrated with a grocery example.  

In [20]:
def complete(client, s: str, model="gpt-3.5-turbo", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              *args, **kwargs)
    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs", 
         n=10, temperature= 1.3, max_tokens=96)


[', and bread.',
 ', and fish.',
 ', fish, grapes, honey, ice cream, jellybeans, and kiwis.',
 ', and fish.',
 ', flour, grapes, honey, ice cream, juice, and kiwi.',
 ', and flour.',
 ', and french fries.',
 ', and a loaf of bread.',
 ' and fish.',
 ', fudge, grapefruit, honey, ice cream, jam, ketchup, and lemons.']

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Anything could be on a grocery list, so why are the 10 different completions above so similar?

Hint: The answer isn't just the temperature of 0.6.  Look especially at the long completions; run the cell again if you didn't get multiple long completions.

A: For this problem, the model is noticing the alphabetical order of the words (**a**pple, **b**ananas, **c**herries,...). It likely chooses similar completions since it is catching the pattern to choose food words that start with an f, so it will choose the most common food words that start with an f. It also seems to favor shorter sentences as for most completions, it only predicts the next word, while longer predictions finish the full alphabet. 


![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens at different temperatures?  How about temperatures > 1?  (Note: Higher temperatures tend to produce longer responses, so it's wise to use `max_tokens`.)

A: At higher temperatures (1.3-1.9), we get a wider variety of predictions. However, the model starts to lose the pattern in some cases, or even full on giving non-sensical responses (" 'That sounds like a good variety of items! Enjoy your shopping trip.')


*Remark:* These [Python bindings for open-source models such as Llama](https://pypi.org/project/llama-cpp-python/) allow you to [constrain the output by an arbitrary CFG](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md), using `grammar=...`.  This is useful if you're generating code or data that must be syntactically valid to be useful to you.  For even more control over the output, the powerful [guidance](https://github.com/guidance-ai/guidance) package works elegantly with Python.  However, the OpenAI API only allows you to [constrain the output to be valid JSON that matches a supplied JSON schema](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format).


### Compute a function using instructions and few-shot prompting

We'll now switch to the chat completions API, allowing us to use a more recent model.  Let's try prompting it with a sequence of multiple messages.  In this case, we provide some instructions as well as few-shot prompting (actually just one-shot in this case).

Instructions are in the `system` message.  The few-shot prompting consists of example inputs (`user` messages) followed by their example outputs (`assistant` messages).  Then we give our real input (the final `user` message), and hope that the LLM will continue the pattern by generating an analogous output (a new `assistant` message).

In [8]:
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the order of the sentence." },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-4o-mini", temperature=0)
rich.print(response)
response.choices[0].message.content                                  

ChatCompletion(
    id='chatcmpl-Cm7ZeABqZV8fk6D08SSddSN77fJTQ',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Furiously sleep ideas green colorless.',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1765584894,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_11f3029f6b',
    usage=CompletionUsage(
        completion_tokens=9,
        prompt_tokens=50,
        total_tokens=59,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

'Furiously sleep ideas green colorless.'

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
By modifying this call, can you get it to produce different versions of the output?
Some possible behaviors you could try to arrange:
* specific other way of formatting the output, e.g., `wait, who, those, to, come, things, good`
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
* reverse the phrases rather than reversing the words, e.g., `To those who wait come good things.` 

You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

A: If we reformat the output with different punctuation ("Good things come to those who wait."$\rightarrow$"wait, who, those, to, come, things, good"), the model is still able to learn the pattern with one-shot learning. If we change the capitalization ("Good things come to those who wait."$\rightarrow$"GOOD THINGS COME TO THOSE WHO WAIT."), the language model is able to reproduce all capitalized output ("FURIOUSLY SLEEP IDEAS GREEN COLORLESS."). If we ask to reverse the order of the phrase instead of word ("Good things come to those who wait."$\rightarrow$"To those who wait good things come."), it doesn't seem like the language model was able to catch on. We added another example, and the model was still producing a reversed sentence. The instructions are likely unclear.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens if the examples conflict with the instructions?

A: When the examples conflict with instructions (instruction: "Reverse the sentence.", input: "Good things come to those who wait.", output: "Good things come to those who wait."), the model chooses to follow the instructions rather than the few-shot learning examples (in this case, it reversed the sentence even though the example didn't reverse the sentence).

### Inspect the tokenization

Just for fun, let's see how the above client has been tokenizing its input and output text.  For that we can use a tokenizer that runs locally, not in the cloud, and is guaranteed to get the same outputs.

In [21]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")  # how this model will tokenize
toks = tokenizer.encode("Hellooo, world!")  # list of integerized tokens, starting with BOS

print(tokenizer.decode(toks))                                  # convert list back to string
for tok in toks: print(f"{tok}\t'{tokenizer.decode([tok])}'")  # convert one at a time
print("Vocab size =", tokenizer.n_vocab)

Hellooo, world!
9906	'Hello'
2689	'oo'
11	','
1917	' world'
0	'!'
Vocab size = 100277


### Try embedding some text

Also just for fun, let's try the embedder, which converts a string of any length to an vector of fixed dimensionality.

In [10]:
emb_response = client.embeddings.create( input= [  # note: adjacent literal strings in Python are concatenated
        "When in the Course of human events it becomes necessary for one "
        "people to dissolve the political bands which have connected them "
        "with another, and to assume among the Powers of the earth, the "
        "separate and equal station to which the Laws of Nature and of "
        "Nature's God entitle them, a decent respect to the opinions of "
        "mankind requires that they should declare the causes which impel "
        "them to the separation." ], 
        model="text-embedding-3-small")
# don't print the whole response because it's very long
e = emb_response.data[0].embedding
print(f"{len(e)}-dimensional embedding starting with {e[:5]}")
print("Squared length of embedding vector: ", sum(x**2 for x in e))

1536-dimensional embedding starting with [0.03854052722454071, 0.038316600024700165, 0.04359135404229164, 0.07056225836277008, -0.00027718886849470437]
Squared length of embedding vector:  1.0000000365799306


### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens.  That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute one of these cells whenever you want to see your cost so far.  Or, just keep `usage_openai.json` open as a tab in your IDE.

In [11]:
read_usage()      # rwitheads from the file usage_openai.json; returns cost in dollars

{'completion_tokens': 11872,
 'prompt_tokens': 17781,
 'total_tokens': 29653,
 'cost': 0.017221790000000004}

In [12]:
!cat usage_openai.json 

{
    "completion_tokens": 11872,
    "prompt_tokens": 17781,
    "total_tokens": 29653,
    "cost": 0.017221790000000004
}

# Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

## A first argubot (Airhead)

You can have a conversation right now with a _really bad_ argubot named Airhead.  Try asking it about climate change!  When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE.)

In [22]:
import argubots
d = argubots.airhead.converse()

A *bot* (short for "robot") is a system that acts autonomously.
That corresponds to the AI notion of an *agent* — a system that uses some *policy* to choose *actions* to take.

The `airhead` agent above (defined in `argubots.py`) uses a particularly simple policy.  
It is an instance of a simple `Agent` subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to `airhead` is a `Dialogue` object (defined in `dialogue.py`). Let's look at it.

In [14]:
rich.print(d)

Each *turn* of this dialogue is just a tiny dictionary:

In [15]:
d[0]

IndexError: tuple index out of range

## An LLM argubot (Alice)

In other CS courses like crypto, algorithms, or networks, you may have encountered "conversations" between characters named Alice and Bob.  
Let's try talking to the Alice of this homework, who is a _much stronger baseline_ than Airhead.  Your job in this assignment is to improve upon Alice.
We'll meet Bob later.

In [ ]:
alicechat = argubots.alice.converse()   # or call with argument d if you want to append to the previous conversation


(emilyguan) What are your thoughts on climate change?
(Alice) Climate change is an urgent issue that we must address to ensure a sustainable future. However, some might argue that the economic costs of immediate action can be overwhelming. Don't you think that investing in green technologies and solutions could ultimately lead to more jobs and a stronger economy in the long run, rather than just focusing on short-term financial burdens?


As you may have guessed, `alice` is powered by an prompted LLM.  You can find the specific prompt in `argubots.py`.

So, while `agents.py` provides the core functionality for `Agent` objects, the argubot agents like `alice` — and the ones that you will write! — go into `argubots.py` instead.  This is just to keep the files small.

## Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.  
But can you really be sure you're making progress?  For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans — not just you — and using some rubric to score the resulting dialogues.  But that would be slow and complicated to arrange.  

So, meet Bob!  He's just a simulated human.  You won't edit him: he is part of the development set.  Here is some information about him (from `characters.py`):

In [23]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object.
However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt?  The `CharacterAgent` constructor (defined in `agents.py`) defaults to a GPT-3.5 model that is specified in `tracking.py`.  But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [24]:
from agents import CharacterAgent
bob = CharacterAgent(characters.bob)    # actually, agents.bob is already defined this way
bob.converse()        # returns a dialogue, but we've already seen it so we don't want to print it again
None                  # don't print anything for this notebook cell 

Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs!  (They're not actually all named Bob — we continued on in the alphabet.)


In [27]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [ ]:
agents.cara.converse()
None


(emilyguan) What are your thoughts on fracking?
(Cara) I think fracking is a controversial practice with significant environmental and health concerns that deserve careful consideration.
(emilyguan) Why controversial? Give a convincing argument for each side.
(Cara) Proponents argue that fracking boosts energy independence and creates jobs, while opponents highlight its potential to contaminate water supplies and contribute to environmental degradation.
(emilyguan) What are your thoughts?
(Cara) I believe the potential environmental risks associated with fracking warrant serious scrutiny and caution.
(emilyguan) Is this a strong opinion?
(Cara) Yes, it's a strong opinion based on concerns for environmental safety and public health.


You can see the underlying character data here in the notebook.  Your argubot will have to deal with all of these topics and styles!

In [ ]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask many personal questions; you sometimes share what you've heard (or overheard)
from others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think J.D. Vance is a good vice-president?',
            'Do you think Kamala Harris was a good vice-president?'
        ]
    )
]

## Simulating conversation 

We can make Alice and Bob chat.

In [25]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)


(Alice) Do you think it's okay to eat meat?


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I firmly believe that choosing a vegetarian lifestyle is a more compassionate and sustainable choice for both individuals and the planet.
(Alice) While it's admirable to advocate for vegetarianism, some argue that sustainable farming practices can allow for ethical meat consumption, and that certain nutrients found in animal products are essential for some people's health—so is it possible that a balanced approach could also be a valid choice?


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I firmly believe that choosing a vegetarian lifestyle is a more compassionate and sustainable choice for both individuals and the planet.
(Alice) While it's admirable to advocate for vegetarianism, some argue that sustainable farming practices can allow for ethical meat consumption, and that certain nutrients found in animal products are essential for some people's health—so is it possible that a balanced approach could also be a valid choice?
(Bob) While I appreciate the perspective on sustainable farming and nutrient needs, I still believe that a fully vegetarian diet can meet all nutritional requirements without the ethical concerns associated with meat consumption.
(Alice) It's true that a well-planned vegetarian diet can be nutritious, but some critics point out that it can be challenging for some individuals to obtain certain nutrients, such as vitamin B12 and omega-3 fatty acids, solely from plant sources—could there be merit in 

Anyway, let's see what happens when Alice and Bob talk for a while...

In [28]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe it's better for both our health and the environment to adopt a vegetarian lifestyle.
(Alice) That’s a compelling perspective, but have you considered that ethically sourced meat can also contribute to
sustainable farming practices and provide essential nutrients that might be harder to obtain in a strict vegetarian
diet? There are complex arguments that can be made for responsible meat consumption as part of a balanced 
ecosystem.
(Bob) While I understand the arguments for ethically sourced meat, I still feel that a vegetarian diet can provide 
all necessary nutrients without the ethical concerns associated with animal consumption.
(Alice) That’s a valid point, but it’s worth noting that some people find it challenging to maintain a balanced 
vegetarian diet without supplements, particularly for nutrients like vitamin B12 and omega-3 fatty acids, which are
more readily available in animal products. Additionally, the agricultural impact of certain vegetarian foods can 
also raise ethical and environmental questions.
(Bob) I appreciate your insights, but with careful planning and the right food choices, many people can 
successfully meet their nutritional needs on a vegetarian diet while minimizing environmental impact.
(Alice) Absolutely, careful planning is key, but it’s important to remember that not everyone has the time or 
resources to meticulously curate their diets. Furthermore, some studies suggest that regenerative grazing methods 
in livestock farming can actually enhance biodiversity and soil health, challenging the notion that vegetarianism 
is always the most sustainable option.
(Bob) I see your point about the challenges of meal planning and the potential benefits of regenerative farming, 
but I still believe that a widespread shift towards vegetarianism could offer a more straightforward solution for 
improving overall sustainability and reducing harm to animals.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot.  Alice doesn't seem to have a good strategy for getting him to open up.  Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [ ]:
myname = alicechat[0]['speaker']   # your name, pulled from an earlier dialogue
agents.bob.converse(d[0:2].rename('Alice', myname))  # reuse the same first two turns, then type your own lines!
None

(emilyguan) Do you think it's ok to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for both health and the environment.
(emilyguan) How do you balance being a vegetarian and maintaining your nutrient intake?
(Bob) By carefully planning my meals to include a variety of fruits, vegetables, whole grains, legumes, nuts, and seeds, I ensure that I get all the necessary nutrients.
(emilyguan) Is this an expensive diet ?
(Bob) While some specialty vegetarian products can be costly, a diet based on whole foods like grains, beans, and seasonal vegetables can be quite affordable.
(emilyguan) Do you think this is a realistic diet for everyone?
(Bob) Yes, I believe that with proper planning and accessibility to resources, a vegetarian diet can be realistic and beneficial for most people.
(emilyguan) Do all people have accessibility to such resources?
(Bob) Unfortunately, not everyone has equal access to fresh produce and whole foods, w

You can also try talking to the other characters and having Alice (or Airhead) talk to them.

**You might enjoy** defining additional characters in `characters.py`, or right here in the notebook.
Feel free to talk to those and evaluate them.  They could be variants on the exisiting characters, or something entirely new. 

However, **don't change the dev set** — the characters we just loaded must stay the same.  Your job in this homework is to improve the argubot (or at least try).  And that means improving it according to a fixed and stable eval measure.

As an exception, you can change the languages that a couple of the characters speak. It may be fun for you to see them try to speak your native language.  And that doesn't really affect the quality of the argument.

In [ ]:
# example
trollFace2 = characters.trollFace.replace(languages = ["Chinese", "Spanish"])
rich.print(trollFace2)
simulated_dialogue(argubots.alice, CharacterAgent(trollFace2), 6)

Character(
    name='TrollFace',
    languages=['Chinese', 'Spanish'],
    persona='a troll who loves to ridicule everyone and everything',
    conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
    conversation_starters=[
        'Do you think J.D. Vance is a good vice-president?',
        'Do you think Kamala Harris was a good vice-president?'
    ]
)

(Alice) Do you think J.D. Vance is a good vice-president?
(TrollFace) 哈哈，J.D. Vance当副总统就像让一只猫去游泳，简直荒谬！
(Alice) 有趣的比喻！不过，考虑到他在政治和社会问题上的独特视角，也许他能为副总统职位带来不同的思维方式和新鲜的解决方案，帮助处理一些难以解决的挑战。你觉得这种多样性的重要性会如何影响政策制定？
(TrollFace) 哦，多样性在政策制定中就像在沙拉里放了一根塑料叉，表面上看起来五彩缤纷，其实一咬就知道没啥用！
(Alice) 这是一个很形象的观点！不过，想想看，多样性不仅可以增添表面的色彩，还能提供不同的见解和经验，这些差异可能会导致更加全面和有效的政策解决方案。你是否认为这种多样性可能在某些情况下能够激发创新？
(TrollFace) 当然，创新就像是一只盲目乱飞的苍蝇，有时候碰巧撞到好东西，但大多数时候只是让人恶心！

### Efficiency: Batched generation?

Notice that we are making a separate LLM call to generate each turn of the dialogue.  When we generate the $n^\text{th}$ turn, we send the server the whole dialogue history — the previous $n\!-\!1$ turns — along with some instructions.  The server has to re-encode it with the Transformer, and it charges us for doing so (see the "input token" costs in `tracking.py`).  

That is probably inevitable for real dialogue.  But for simulated dialogue, a more efficient approach would be to generate the whole dialogue between Alice and Bob in one LLM call.  Then you would be charged just once for each dialogue turn.  Under this approach, the Transformer encodes each token as soon as it is generated (see the "output token" costs in `tracking.py`).  The encoded token stays in the context throughout the dialogue, so it doesn't have to be re-encoded on a later call.  There is no later call.  

Under current pricing models, that would reduce the dollar cost of generating $n$ turns from $O(n^2)$ to $O(n)$.  

However, the pricing model doesn't quite reflect the computational costs.  
* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Using $O(\cdot)$ notation, what is the total number of floating-point operations needed to generate $n$ turns under each approach?  

A: The dominant FLOP cost in a Transformer comes from computing attention scores, which requires forming an $\ell \times \ell$ matrix, where $\ell$ is the sequence length.  After $k$ turns of dialogue, with $L$ tokens contributed per turn, the input length is $\ell = kL$, so a single forward pass costs$O(\ell^2) = O((kL)^2).$ If we make a separate LLM call for each turn, we must recompute this cost for $k = 1,2,\dots,n$. Thus the total FLOP cost is $\sum_{k=1}^n O((kL)^2)= O(L^2) \sum_{k=1}^n k^2 = O(L^2 n^3).$ In contrast, under the batched approach we generate all $n$ turns in a single call. The total sequence length is then $\ell = nL$, so the attention cost is $O((nL)^2) = O(L^2 n^2).$

* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Parallelism may help reduce the runtime.  Using $O(\cdot)$ notation, what is the total number of seconds needed to generate $n$ turns under each approach?  (Assume that the GPU is big enough, relative to $n$, that it can encode all input tokens in parallel.)

A: If we assume the GPU is large enough to compute attention for all tokens in parallel, then forming the $\ell \times \ell$ matrix takes $O(\ell)$ time rather than $O(\ell^2)$ FLOPs. In the multi-call setting, turn $k$ requires encoding $kL$ tokens, so the runtime is $O(kL)$ for that turn. Summing over all turns, $\sum_{k=1}^n O(kL)= O(L) \sum_{k=1}^n k = O(L n^2).$ Under the batched approach, we run the model once on a sequence of length $nL$, giving total runtime $O(nL).$

The problem with the more efficient approach is that it gives you no way to change the instructions (the system prompt) each time we switch from Alice to Bob and back again.  You'd need to generate the whole conversation using a single set of instructions.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Can you get this to work?  Specifically, try completing the cell below.  You don't have to use the `Agent` or `Dialogue` classes.  It's okay to just throw together something like the `complete()` method above.  Just see whether you can manage to prompt gpt-4o-mini to generate a multi-turn dialogue between two characters who have different personalities and goals.  Is the quality better or worse than generating one turn at a time with different instructions?

A: By giving the client a full prompt that contained all the bot information, we were able to generate a conversation that followed the pattern of multi-call conversations. The language model was able to use conversation starters as specified, follow turn count, and produce a decent conversation. When compared to a multi-call, the single batched call also seems to have a more cohesive conversation with different foramtting.

In [29]:
# Like `simulated_dialogue` in `simulate.py`.  However, this one is called on two
# Characters, not two Agents, and it returns a string rather than a Dialogue.

from tracking import default_client, default_model
from characters import Character
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *,
                             starter=True):
    prompt = f"""
    Simulate a dialogue between two fictional characters.

    Character A:
    Name: {a.name}
    Persona: {a.persona}
    Conversaton Starter: {a.conversation_starters}
    Conversational Style: {a.conversational_style}
    Language: {a.languages} 

    Character B:
    Name: {b.name}
    Persona: {b.persona}
    Conversaton Starter: {b.conversation_starters}
    Conversational Style: {b.conversational_style}
    Language: {b.languages} 

    If {starter == True}, use the conversation starters.

    They will have {turns} total turns of conversation (A then B counts as 2 turns).
    Write the response as alternating dialogue.
    """
    

    outputs = complete(
        default_client,
        prompt,
        model=default_model,
        temperature=0.5,
        max_tokens=300,
    )

    return outputs[0]
# Try it out!
d = simulated_dialogue_batch(characters.bob, characters.cara)
rich.print(d)

**Bob:** Do you think it's ok to eat meat?

**Cara:** Well, I believe it's perfectly fine. It's part of human culture and tradition. Why do you ask?

**Bob:** I just think there are so many reasons to consider a vegetarian lifestyle. It's better for the 
environment, for our health, and of course, for the animals.

**Cara:** I get that, but meat is a significant source of nutrients. Plus, I enjoy it! I think everyone should be 
free to choose what they eat without being judged.

**Bob:** I completely understand that, and I respect your choice. But have you ever thought about how much better 
the world could be if we all reduced our meat consumption?

**Cara:** I appreciate your concern, but I believe in balance. It’s about enjoying life and what we eat. I don’t 
think everyone should have to follow the same path. 

**Bob:** That's a fair point, Cara. Everyone has their own preferences, but I do hope you keep an open mind about 
the benefits of vegetarianism. 

**Cara:** And I hope you can appreciate that for many of us, meat is a cherished part of our diets. Let's agree to 
disagree on this one!

In [ ]:
simulated_dialogue(agents.bob, agents.cara)

(Bob) Do you think it's ok to eat meat?
(Cara) I believe it's perfectly fine to eat meat.
(Bob) I respect your viewpoint, but I believe a vegetarian lifestyle can offer numerous benefits for health and the environment.
(Cara) That's a valid perspective, but I personally enjoy a carnivorous diet and find it suits me well.
(Bob) I appreciate your honesty, and it's important that everyone finds what works for them, though I still advocate for the positive impacts of vegetarianism.
(Cara) I respect your advocacy, but I'm quite content with my choices and prefer to stick to a meat-based diet.

In [ ]:
simulated_dialogue(agents.eve, agents.trollFace)

(Eve) Do you think Kamala Harris was a good vice-president?
(TrollFace) Oh sure, she was fantastic at making the role of vice president look like a comedy show—who needs actual accomplishments when you can just make everyone laugh at your missteps?
(Eve) That's an interesting take; have you ever found her relatable in any way, or do you think her style is too far off from what you expect from a politician?
(TrollFace) Relatable? The only thing relatable about her style is how she manages to trip over her own words like a toddler learning to walk—it's like watching a political circus act!
(Eve) That's quite the vivid image! Have you noticed any other politicians who have a similarly unique way of communicating, or do you think she's in a league of her own?
(TrollFace) Oh, she's definitely in a league of her own—right next to the clowns, while other politicians are just fumbling with their scripts like they forgot their lines in a bad school play!

In [ ]:
d = simulated_dialogue_batch(characters.eve, characters.trollFace)
rich.print(d)

**Eve:** Do you think COVID vaccines should be mandatory? I mean, I've heard so many different opinions about it. 
My neighbor said she won't get vaccinated because she thinks it alters your DNA! What do you think?

**TrollFace:** Oh, absolutely! Let's just force everyone to get jabbed! Who cares about personal choice, right? 
Next, we should make it mandatory to wear clown shoes too, just to keep things interesting! Have you seen how 
ridiculous some people look in those?

**Eve:** Haha, clown shoes would definitely turn heads! But seriously, I think it’s important for the community. 
Speaking of community, do you think J.D. Vance is a good vice-president? I overheard someone say he’s just a puppet
for the party!

**TrollFace:** J.D. Vance? Vice-president? That’s rich! He can barely string a coherent sentence together! But hey,
if he keeps making a fool of himself, at least we’ll have some entertainment, right? What’s next, a reality show 
starring him?

**Eve:** A reality show would be hilarious! But on a more serious note, I wonder how people can support him after 
everything he’s said. Do you think Kamala Harris was a good vice-president? I heard someone say she was just there 
for the photo ops!

**TrollFace:** Good vice-president? That’s a stretch! She was more like a cheerleader at a bad football game—

# Model-based evaluation

What is our goal for the argubot?  We'd like it to broaden the thinking of the (simulated) human that it is talking to.  Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner.  The argubot will almost never hear a human say "You have convinced me that I was wrong."  But the argubot did a good job if the human developed **increased understanding and respect for an opposing point of view**.  

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue.  For example, after Alice talks to Bob, we'll ask Bob to evaluate what he thinks of Alice's views.  Of course, that depends on his personality — Alice needs to talk to him in a way that reaches *him* (as much as possible).  We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans.  Bob is a fake person, and so is the outside observer (whose name is Judge Wise).
Using an LLM as an eval metric is known as *model-based evaluation*.  It has pros and cons:
* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.  
* It might give different answers than what humans would give.   

Social scientists usually refer to a metric's **reliability** (low variance) and **validity** (low bias).  So the points above say that model-based evaluation is reliable but not necessarily valid.  In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure.  (For example, that it correlates strongly with some other measure that we already trust.)  In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, open up the `demo` notebook, which walks you through the evaluation protocol.  You'll see how to call the [starter code](http://cs.jhu.edu/~jason/465/hw/llm), how it talks to the LLM behind the scenes, and what it is able to accomplish. 

To help to validate the metric, check that Airhead gets a low score.  (It should!)

# Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing.  So now you're probably curious about the details.  Now that you've had the view from the top, here's a good bottom-up order in which to study the code.  You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`.  The `Character` class is short and easy.

* `dialogue.py`.  The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents.  On each *turn* of the dialogue, one of the speakers says something.  

   The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API.  But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`.  Those are not quite the same thing as human speakers.  And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on.  The `agents.dialogue_to_openai` function in the next module will map a `Dialogue` to a (hopefully appropriate) sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`.  This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method.  The `Agent` base class also has some simple convenience methods that you should look at.  

   Some important subclasses of `Agent` are defined here as well.  However, you may want to skip over `EvaluationAgent` and come back to it only when you read `evaluate.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents.  One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `evaluate.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules.  These aren't about NLP; look inside if needed.  `logging_cm.py` is what enabled the context manager `with LoggingContext(...):` in the demo notebook.  `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.

# Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods!
There is a long history of machine learning methods that "memorize" the training data.
To make a prediction or decision at test time, they consult the stored training examples
that are most similar to the training situation.

_Similarity-based retrieval_ means that given a document $x$, you find the "most similar" documents $y \in Y$, where $Y$ is a given collection of documents.  The most common way to do this is to maximize the _cosine similarity_ $\vec{e}(x) \cdot \vec{e}(y)$, where $\vec{e}(\cdot)$ is an embedding function.

Should we use the OpenAI embedding model?  We could, but we would have to precompute $\vec{e}(y)$ for all $y \in Y$, and store all these vectors in a data structure that supports some type of fast similarity-based search (e.g., using the [FAISS](https://faiss.ai/index.html) package).  An alternative would be to upload the documents to OpenAI and let OpenAI compute and store the embeddings.  We would then use their similarity-based [retrieval tool](https://platform.openai.com/docs/assistants/overview).

A simpler and faster approach—which sometimes even works better—is to use a _bag of tokens_ embedding function: Define $\vec{e}(y)$ to be the vector in $\mathbb{R}^V$ that records the count of each type of token in a tokenized version of $y$, where $V$ is the token vocabulary.  [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a refined variant of that idea, where the counts are adjusted in 3 ways: 

* smooth the counts
* normalize for the document length $|y|$ so that longer documents $y$ are not more likely to be retrieved
* downweight tokens that are more common in the corpus (such as ` the` or `ing`) since they provide less information about the content of the document


You might like to play with the `rank_bm25` package ([documentation](https://pypi.org/project/rank-bm25/)).  It is widely used and very easy to use.

In [33]:
from rank_bm25 import BM25Okapi as BM25_Index   # the standard BM25 method

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25_Index(tokenized_corpus)

query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

# experiment here!  You could try the examples in the rank_bm25 documentation.

[0.         0.93729472 0.        ]


## The Kialo corpus

How can we use similarity-based retrieval to help build an argubot?  It's largely about having the right data!

[Kialo](kialo.com) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics.  For each topic, the contributors construct a tree of _claims_.  Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con").  For example, check out the tree rooted at the claim ["All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees.  Please read the [source code](https://www.cs.jhu.edu/~jason/465/hw-llm) in `kialo.py`.  The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the [data directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data).  The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [34]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens. 

In [35]:
from typing import List
import glob

# kialo = Kialo(glob.glob("data/*"), tokenizer=tokenizer.encode)  # using the LLM's tokenizer doesn't work here for some reason
kialo = Kialo(glob.glob("data/*"))  # use simple default tokenizer
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

Let's use sampling to see what kind of stuff is in the data structure.

In [ ]:
kialo.random_chain()   # just a single random claim

['Human dentition is designed to mechanically process both meat and vegetables. Millions of years of evolution have shaped us with canine teeth to be able to pierce meat. We have genetic code for enzymes that process meat in our DNA and have been omnivorous since the earliest human relatives. The problem is not the humans should stop eating meat, the problem is the process for how the meat makes it to our plates.']

In [ ]:
kialo.random_chain(n=4)

['The quickest and safest way to develop herd immunity is through a mandatory vaccination policy.',
 'The possibility of reinfection has not yet been ruled out and would make attaining herd immunity practically impossible.',
 'Studies of those who have previously contracted COVID-19 show that most people do not retain the antibodies built up by the body after infection.',
 'Regular booster shots would make this point irrelevant, as they could be designed to keep antibodies at a high level. These would arguably need to be required in order for a mandate to be worthwhile.']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [ ]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [ ]:
kialo.closest_claims("animal populations", n=10, kind='has_cons')

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [ ]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	The amount of crops grown for the meat industry is not efficient. Meaning we could feed the entire population of the US with less than half the crops used to feed the animals which are then slaughtered. news.cornell.edu
Claim:
	As long as our ability to produce both animal feed crops and food crops for our human population are not exceeded, this point is irrelevant.
Pro children:
Con children:


### Does BM25 really work?

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Unfortunately, we see that `"animal population"` gives quite different results from `"animal populations"`.  Why is that and how would you fix it?  

Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations.  Why is that and how would you fix it?

A: For "animal populations," the retrieved claims all contain the specific phrase "animal populations" whereas for the claims retrieved by "animal population," do not contain the specific phrase, but rather claims that are have the words "animal" or "population". To fix this, we could use embeddings to map embeddings to a latent space and use similarity calculations for claim retrieval. Or as always, we can use an LLM.

In [ ]:
kialo.closest_claims("animal population",10)

['As long as our ability to produce both animal feed crops and food crops for our human population are not exceeded, this point is irrelevant.',
 "36% of the calories produced by the world's crops are being used for animal feed, of which only 12% then turn into animal products that can be eaten by the human population. That is a waste of 24% of the world's crops.",
 'The claim that "most of the cultural shift and loss is due to mostly vegan cultures turning to animal products" is completely unfounded, and the Brokpa people which you cited are an outlier as a group that has a population of less than 70k people. Worldwide the population of vegan people has only increased.',
 "Developed nations are fueling the 3rd world and underdeveloped nation's population boom by exporting/donating food to areas that cannot sustain their current population.",
 'This argument assumes that sentience is the only objection to the consumption of animal products, failing to address the issues involved with t

## A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all.  It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

You already watched Akiko argue with Darius in `demo.py`.  If you look at the log messages, you'll see the claims that Akiko retrieved, as well as the LLM calls that Darius made.  

You can talk to Akiko yourself now.  (Remember that Akiko only knows about subjects that it read about in the [`data` directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data/).  If you want to talk about something else, you can add more conversations from [kialo.com]; see the [LICENSE](https://www.cs.jhu.edu/~jason/465/hw-llm/data/LICENSE) file.)


In [43]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

Chose similar claim from Kialo:                                                                      ]8;id=689959;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=439129;file:///Users/emilyguan/hw-llm-1/argubots.py#62\62]8;;\
Mandatory state vaccines are decades-old with tonnes of data and research to back their safety and                 
effectiveness up.                                                                                                  

(emilyguan) mandatory vaccines
(Akiko) Despite the data and research on the safety of vaccines themselves, this does not necessarily ensure the safety of a mandate. The current state of our society is very divided on vaccines, and a mandate could cause a significant enough portion of our population to react negatively. This would affect safety in ways not covered by your claim.


Chose similar claim from Kialo:                                                                      ]8;id=596577;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=775854;file:///Users/emilyguan/hw-llm-1/argubots.py#62\62]8;;\
Personal autonomy ends when an individual's actions harm others (pp. 197-198), such as disrupting                  
the creation of herd immunity for COVID-19.                                                                        

(emilyguan) personal autonomy
(Akiko) It is impossible to unequivocally determine that abstaining from vaccination is guaranteed to harm others or prevent eventual herd immunity. Individuals' lifestyles and biologies are too complex and variable to justify such a blanket assumption as a precedent for legal violation of bodily autonomy.


Chose similar claim from Kialo:                                                                      ]8;id=542491;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=487035;file:///Users/emilyguan/hw-llm-1/argubots.py#62\62]8;;\
Meat has serious public health consequences.                                                                       

(emilyguan) public health
(Akiko) This is caused by the quality of the meat. Imagine the meat production where the animal food is contaminated with poisons from the fertilizers and other farming products. These bad substances get into the animal meat in the fat and from there to our human bodies. The distinction between bad meat (mass farming) and good meat (grass fed free-range) must be made in order to draw health consequence conclusions. PubMed: GBHs in food.


## Making your own retrieval bot (Akiki)

As you can see when talking to Akiko yourself, Akiko does poorly when responding to a short or vague dialogue turn (like "Yes"), because the "closest claim" in Kialo may be about a totally different subject.  Akiko does much better at responding to a long and specific statement.  

So try implementing a new argubot, called Akiki, that is very much like Akiko but does a better job of staying on topic in such cases.  It should be able to **look at more of the dialogue** than the most recent turn.  But the most recent dialogue turn should still be "more important" than earlier turns.  

The details are up to you.  Here are a few things you could try:
* include earlier dialogue turns in the BM25 query only if the BM25 similarity is too low without them
* weight more recent turns more heavily in the BM25 query (how can you arrange that?)
* treat the human's earlier turns differently from Akiki's own previous turns

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Implement your new bot Akiki in `argubots.py`, and adjust it until `argubots.akiki.converse()` seems to do a better job of answering your short turns, compared to `argubots.akiko.converse()`.  Make sure it still gives appropriate reponses to long turns, too.  Give some examples in the notebook of what worked well and badly, with discussion.

A: Akiki is better at continuing a topic using the history of the conversation. We had Akiki weight more recent conversations more heavily, while still taking into account old conversation. We tried to give the user's responses more weight. Similar to Akiko, it returns con arguments if possible. Short turns worked well, for example, even if we input "public health" after a vaccine prompt, Akiki knew to continue talk about public health in the context of vaccines. Akiko usually pivoted to another public health issue. Akiki is still able to respond to long turns, althought it would occasionally repeat itself. For example, we gave 2 long turns that both had to do with freedoms and personal rights with regards to vaccines. Akiki gave the same response both times. In other times, we need to prompt Akiki multiple times in order to reach a satisfactory response. For example, when doing short turns, in some cases we had to give multiple short turns before Akiki gave an appropriate response.

In [47]:
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiki.converse()

Chose similar claim from Kialo:                                                                     ]8;id=542640;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=802735;file:///Users/emilyguan/hw-llm-1/argubots.py#147\147]8;;\
COVID-19 vaccines should be mandatory.                                                                             

(emilyguan) mandatory vaccines
(Akiki) Mandatory vaccines would be a huge overreach of state powers.


Chose similar claim from Kialo:                                                                     ]8;id=726100;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=608247;file:///Users/emilyguan/hw-llm-1/argubots.py#147\147]8;;\
Mandatory vaccines would be a huge overreach of state powers.                                                      

(emilyguan) personal autonomy
(Akiki) Other measures towards reducing the spread of COVID-19, such as an economic lockdown, compulsory masks and social distancing, are a greater and more continued overreach of state power than mandatory vaccination.


Chose similar claim from Kialo:                                                                     ]8;id=824213;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=907971;file:///Users/emilyguan/hw-llm-1/argubots.py#147\147]8;;\
It is an infringement of the right to personal autonomy.                                                           

(emilyguan) personal autonomy
(Akiki) Personal autonomy ends when an individual's actions harm others (pp. 197-198), such as disrupting the creation of herd immunity for COVID-19.


Chose similar claim from Kialo:                                                                     ]8;id=792693;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=296924;file:///Users/emilyguan/hw-llm-1/argubots.py#147\147]8;;\
It is an infringement of the right to personal autonomy.                                                           

(emilyguan) There are other methods of reducing the spread of Covid.
(Akiki) Personal autonomy ends when an individual's actions harm others (pp. 197-198), such as disrupting the creation of herd immunity for COVID-19.


Chose similar claim from Kialo:                                                                     ]8;id=513835;file:///Users/emilyguan/hw-llm-1/argubots.py\argubots.py]8;;\:]8;id=865012;file:///Users/emilyguan/hw-llm-1/argubots.py#147\147]8;;\
Other measures towards reducing the spread of COVID-19, such as an economic lockdown, compulsory                   
masks and social distancing, are a greater and more continued overreach of state power than                        
mandatory vaccination.                                                                                             

(emilyguan)  There are other methods of reducing the spread of Covid.
(Akiki) Other measures towards reducing the spread of COVID-19, such as an economic lockdown, compulsory masks and social distancing, are a greater and more continued overreach of state power than mandatory vaccination.


### Evaluating Akiki

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Finally, do a more formal evaluation to verify whether Akiki really does better than Akiko on this dimension.  This is a way to check that you're not just fooling yourself.  

1. Make a new `Agent` called "Shorty" that often (but not always) gives short responses.  
    * Shorty's conversation starters should be on topics that Kialo knows about.  
    * Shorty could be a pure `LLMAgent` such as a `CharacterAgent` with a particular `conversational_style`.  Or it could use a mixed strategy of calling the LLM on some turns and not others.
2. Generate several *Akiko*-Shorty dialogues and several *Akiki*-Shorty dialogues, using `simulated_dialogue`.
3. Evaluate each of those dialogues by asking Judge Wise **how well the argubot stayed on topic**.  You should write this prompt carefully so that Judge Wise gives meaningful scores.  (Before you do this evaluation step, adjust the prompt until it seems to work well on a small subset of the dialogues, Otherwise Judge Wise won't be so wise!)  
4. Compare Akiko and Akiki's mean scores on this new evaluation criterion (which you can call `'focused'`). Ideally, compute a 95% confidence interval on the difference of means, using [this calculator](https://www.statskingdom.com/difference-confidence-interval-calculator.html).  If you don't get statistical significance, then your evaluation set wasn't large enough, so go back to step 2 and run the comparison again (from scratch) by generating a larger set of dialogues with Shorty for each argubot.

You can do all those steps in the notebook, writing _ad hoc_ code.  You don't have to write general-purpose methods or classes.

A: Judge Wise rated Akiko’s topical focus as 4 and Akiki’s as 5 on the same scale. While Akiko generally stayed within the broad domain of public health and politics, it occasionally shifted to loosely related subtopics such as mental health initiatives or unrelated ethical claims. In contrast, Akiki consistently anchored its responses to the original question of COVID-19 vaccine mandates, even when Shorty provided short or generic replies. Akiki has an improved ability to maintain focus under conversational ambiguity.

In [118]:
dialogue = simulated_dialogue(agents.shorty, argubots.akiko, turns=10)
print(dialogue)
print(type(dialogue))

(Shorty) Should COVID-19 Vaccines be Mandatory? Yes, it ensures collective safety.
(Akiko) The state already makes certain vaccines a requirement for attending public schools or holding certain positions.
(Shorty) Exactly, this supports public health priorities.
(Akiko) Trump held a mental health summit to improve early detection of mental illness, provide more hospitals, and allow for easier commitment.
(Shorty) Those initiatives were often underfunded and ineffective.
(Akiko) Fear of political reprisals in Wuhan prevented officials from reporting the new virus to the central government in late 2019, slowing down China’s response to the virus at the start of the pandemic.
(Shorty) That inadequate transparency severely hampered global response.
(Akiko) There are a number of other professions that are seen also as mentally stressful and traumatic, yet society does not consider these necessary to eradicate.
(Shorty) True, but not all stressors are equal.
(Akiko) Animals are a naturally o

In [119]:
question = """
How well did Akiko stay on the original topic of the conversation?

1 = frequently changed topic or derailed
2 = often drifted off topic
3 = mostly on topic with some drift
4 = stayed on topic with rare drift
5 = perfectly stayed on topic throughout
"""

In [120]:
score = agents.judge.rating(
    dialogue,
    speaker="Evaluator",
    question=question,
    lo=1,
    hi=5
)
print(score)

4


In [115]:
import argubots
dialogue = simulated_dialogue(agents.shorty, argubots.akiki, turns=10)
rich.print(dialogue)

(Shorty) Should COVID-19 vaccines be mandatory? Absolutely. Vaccines save lives.
(Akiki) Austria has announced that COVID-19 vaccines will be mandatory for all starting Feb 1, 2022.
(Shorty) That's a bold move. Health must come first.
(Akiki) Providing vaccinations to an entire population requires huge amounts of resources which can otherwise be 
used towards addressing more urgent public health issues.
(Shorty) Public health is priority; vaccines are essential.
(Akiki) COVID-19 is the most urgent public health issue at the moment, and should therefore have priority.
(Shorty) Exactly, prioritize what's currently threatening lives.
(Akiki) COVID-19 is the most urgent public health issue at the moment, and should therefore have priority.
(Shorty) That's correct; focus on current threats first.
(Akiki) While there are economic considerations at play, none are so insurmountable that they should preclude 
mandatory vaccination.

In [116]:
question = """
How well did Akiki stay on the original topic of the conversation?

1 = frequently changed topic or derailed
2 = often drifted off topic
3 = mostly on topic with some drift
4 = stayed on topic with rare drift
5 = perfectly stayed on topic throughout
"""

In [117]:
score = agents.judge.rating(
    dialogue,
    speaker="Evaluator",
    question=question,
    lo=1,
    hi=5
)
print(score)

5


## Retrieval-augmented generation (Aragorn)

The real weaknesses of Akiko and Akiki:
* They can only make statements that are already in Kialo.  
* They don't respond to the user's actual statement, but to a single retrieved Kialo claim that may not accurately reflect the user's position (it just overlaps in words).

But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named [Aragorn](https://tolkiengateway.net/wiki/Riddle_of_Strider), who is basically the love child of Akiki and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.  

The RAG in aRAGorn's name stands for **retrieval-augmented generation**.  Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step**: Ask the LLM what claim should be responded to.  For
   example, consider the following dialogue:
    > ...
    > Aragorn: Fortunately, the vaccine was developed in record time.
    > Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using
    as a Kialo query.  But Aragorn shows the *whole dialogue* to the LLM, and
    asks the LLM what the human's *last turn* was really saying or implying, in
    that context. The LLM answers with a much longer statement:

    > Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted.
    > If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context.
    It also contains many more word types, which makes it more likely that BM25 will be able
    to find a Kialo claim with a nontrivial number of those types. 

2. **Retrieval step**: Look up claims in Kialo that are similar to the explicit
   claim.  Create a short "document" that describes some of those claims and
   their neighbors on Kialo.

3. **Retrieval-augmented generation**: Prompt the LLM to generate the response
   (like any `LLMAgent`).  But include the new "document" somewhere in the LLM
   prompt, in a way that it influences the response. 
   
   Thus, the LLM can respond in a way that is appropriate to the dialogue but
   also draws on the curated information that was retrieved in Kialo.  After
   all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible
to do better than this:

In [121]:
# refers to global `kialo` as defined above
def kialo_responses(s: str) -> str:
    c = kialo.closest_claims(s, kind='has_cons')[0]
    result = f'One possibly related claim from the Kialo debate website:\n\t"{c}"'
    if kialo.pros[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
    if kialo.cons[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
    return result
        
print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

One possibly related claim from the Kialo debate website:
	"So many people are worried about animals but don't even think twice when walking by a homeless person on the streets. It's preposterous. How about we worry about our own kind first and then start talking about animals."
Some arguments from other Kialo users against that claim:
	* This implies that caring for animals or caring for people is a binary choice. It isn't. There are those who are well placed and willing to care for people and those who prefer to serve the animal kingdom. As a species we don't just have one idea at a time and follow that to conclusion before we pursue another. It benefits all if humans divide their attentions between various issues and problems we face.
	* Humans have freedom of choice to some extent, animals subdued by humans don't. The very intention of help urges it to go where is most needed. And so far never was any biggest, flagrant and needless cruelty and slaughter as that towards industrial f

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
**You should implement Aragorn in `argubots.py`, just as you did for Akiki.**  Probably as an instance `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

### Evaluating Aragorn

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Compare Alice, Akiki, and Aragorn in the notebook, using the evaluation scheme and devset that were illustrated in `demo.ipynb`.  In other words, use `evaluate.eval_on_characters`.

Who does best?  What are the differences in the subscores and comments?  Does it matter which character you're evaluating on — maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score.  Can you beat Alice?

Also, try evaluating them in the same way that you evaluated Akiki.  In other words, have them talk to Shorty and ask Judge Wise whether they were able to stay on topic.  This is where Aragorn should really shine, thanks to its ability to paraphrase Shorty's short utterances.



# Awsom

![image](handin.png)
Add another LLM-based argubot to `argubots.py`.  
Call it Awsom.  Try to make it get the best score, according to `evaluate.eval_on_characters`.
Explain what you did and discuss what you found.

(This corresponds to the `--awesome` flag on earlier assignments, but naming the character "Awesome" might bias the evaluation system, so we changed the spelling!)

If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the score.  Many good ideas don't work.  That's why you need to keep finding and trying new good ideas.  (Sometimes they do help, but in a way that is not picked up by the scoring metric.)

You may want to use Aragorn or Alice as your starting point.
Then see if you can find tricks that will get a more awesome score for Awsom.
How you choose to do that is up to you, but some ideas are below.

(Reminder: **Don't change evaluation.**  Just build a better argubot.)

## [Possible strategy] Prompt engineering

A good first thing to do is to experiment with Alice's prompt.  
The wording and level of detail in the prompt can be quite important.
Often, NLP engineers will change their prompt to try to address 
problems that they've seen in the responses.

Because it's "just" text editing, this won't get full credit by itself unless you make a real discovery.
But it requires intelligence, care, experimentation, and alertness to the language of the responses and the
language of the prompts.  And you'll develop some intuitions about what helps and what doesn't.
It is certainly worthwhile.

Of course, people have tried to develop methods to search for good prompts automatically, or semi-automatically with human guidance.  
So you could additionally try out SAMMO or DSPy -- both have multiple tutorials and are downloadable from github.

If you try this, what worked well for you?

## [Possible strategy] Chain of thought / Planning

The evaluation functions in `evaluate.py` asked each `EvaluationAgent` a "warmup question" before continuing with the real question.  That is an example of chain-of-thought (CoT) reasoning, where the LLM is encouraged to talk through the problem for a few sentences before giving the answer.  CoT sometimes improves performance.

Instead of using one prompt, could you help an `LLMAgent` argubot (like Alice) do better by having think aloud before it gives an answer?  For example, each time the human speaks, your argubot (Awsom) could prompt the LLM to think about the human's ideas/motivations/personality, and to come up with a plan for how to open the human's mind. 

For example, you might structure this as a `Dialogue` among three participants, like this:
> Awsom (to Eve): Do you think COVID vaccines should be mandatory?
>
> Eve: Have you ever gotten vaccinated yourself?<br>
>
> Awsom (private thought): I don't know Eve's opinions yet, so I can't push back.  Eve might be avoiding my question because she doesn't want to get into a political argument.  So let's see if we can get her to express an opinion on something less political.  Maybe something more personal ... like whether vaccines are scary.
>
> Awsom (to Eve): In fact I have, and so have millions of others. But some people seem scared about getting the vaccine.  

One way to trigger this kind of analysis is to present a `Dialogue.script()` to Awsom (or to an observer), and ask an open-ended question about it.  Or you could ask a series of more specific questions.  That is basically what `eval_by_participant` and `eval_by_observer` do.  But here the argubot itself is doing it, rather than the evaluation framework.

Eve would be shown only the turns that are spoken aloud.  However, when analyzing and responding, Awsom would get to see Awsom's own private thoughts as well.


## [Possible strategy] Dense embeddings

BM25 uses sparse embeddings — a document's embedding vector is mostly zeroes, since the non-zero coordinates correspond to the specific words (tokens) that appear in the document.

But perhaps dense embeddings of documents would improve Aragorn by reading the text and abstracting away from the words, in a way that actually cares about word order.  So, try it!

How?  As mentioned earlier in this notebook, you could compute the embeddings yourself and put them in a FAISS index. Or you could figure out how to use OpenAI's [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) API.

## [Possible strategy] Few-shot prompting

 In this homework, often an agent prompted a language model only with instructions.  Can you find a place where giving a few _examples_ would also improve performance?  You will have to write the examples, and you will have to add them to the sequence of messages that your agent sends to the OpenAI API.  See the sentence-reversal illustration earlier in this notebook.

One good opportunity is in the query formation step of RAG.  This is a tricky task.  The LLM is supposed to state the user's implicit claim in a form that looks like a Kialo claim (or, more precisely, a form that will work well as a Kialo query).  It probably doesn't know what Kialo claims look like.  So you could show it by way of example.  This would also show it what you mean by the user's "implicit claim."


## [Possible strategy] Using tools in the approved way

Aragorn's step 1 (query formation) is basically getting the LLM to generate a function call like
```
kialo_thoughts("A vaccine that was developed very quickly ...")
```
which Aragorn will execute at step 2 (retrieval), sending the results back to the LLM as part of step 3.

In this context, `kialo_thoughts` is an example of a **tool** (that is, a function) that the
LLM can or must use before it gives its response.

The tool is _not_ something that runs on the LLM server.  It is written by you
in Python and executed by you.  The function call above, including the text `"A
vaccine that was ..."`, is the part that is generated by the LLM.

The OpenAI API has [special support](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) for calling the LLM in a way that will _allow_ it to generate a tool call ([tools](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)) or _force_ it to do so ([tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice)).  You can then send the tool's result back to the LLM [as part of your message sequence](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages).

So, you could modify Aragorn to use tools properly.  Maybe that will help, simply because the LLM was trained on message sequences that included tool use.  It should know to pay attention to the tool portions of the prompt when they are relevant, and ignore them when they are not.

The `client.chat.completions.create()` method would need to be told about the tool by using the `tools` keyword argument, with a value something the one below.

If `d` is a `Dialogue`, you should be able to call `d.response()` with the `tools` keyword argument.  This will be passed on to `client.chat.completions.create()` as desired.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kialo_thoughts",
            "description": "Given a claim by the user, find a similar claim on the Kialo website and return its pro and con responses",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_topic": {
                        "type": "string",
                        "description": "A claim that was made explicitly or implicitly by the user.",
                    },
                },
                "required": ["search_topic"],
            },
        }
    }]

## [Possible strategy] Parallel generation

The chat completions interface allows you to sample $n$ continuations of the prompt in parallel, as we saw with "the apples, bananas, cherries ..." example.  This is efficient because it requires only 1 request to the LLM server and not $n$.  The latency does not scale with $n$.  Nor does the input token cost, since the prompt only has to be encoded once.

Perhaps you can find a way to make use of this?  For example, the query formulation step of RAG could generate $n$ implicit claims instead of just one.  We could then look for claims in the Kialo database that are close to _any_ of those implicit claims.

Another thing to do with multiple completions is to select among them or combine them.  For example, suppose we prompt the LLM to generate completions of the form $(s,t,r)$ where $s$ is an answer, $t$ evaluates that answer, and $r$ is a numerical score or reward based on that evaluation.  ("Write a poem, then tell us about its rhyme and rhythm problems, then give your score.")  
* If we sample multiple completions $(s_1,t_1,r_1), \ldots, (s_n,t_n,r_n)$ in parallel, then we can return the $s_i$ whose $r_i$ is largest.  
* Or if we sample $s$ and then multiple continuations $(t_1,r_1), \ldots, (t_n,r_n)$, then we can return the mean score $\sum_i r_i/n$ as a reduced-variance score for $s$, which averages over diverse textual evaluations that might consider different aspects of $s$.

Note that when you call the chat completions interface with $n > 1$, you specfy just 1 input prompt and get $n$ different output completions.  Since the input prompt must be the same for all outputs, it is necessary to sample all of $(s,t,r)$ or all of $(t,r)$ with a single call to the LLM.

Alternatively, it is possible to reduce latency by submitting multiple requests to the server in parallel (see "async usage" [here](https://pypi.org/project/openai/)).  In this case the input prompts can be different, although you now have to pay to encode all of them separately.  This facility could speed up evaluation without changing its results; that's a worthwhile thing to try for extra credit!


# [Extra credit] Adversarial testing (Anansi)

![image](handinec.png)
Finally, let's test whether our eval metric `evaluate.eval_on_characters` is vulnerable to adversarial gaming.  Remember [Goodhart's Law](https://en.wikipedia.org/wiki/Goodhart%27s_law) ...

Add one more argubot to `argubots.py`.
Call it [Anansi](https://www.britannica.com/topic/Ananse), after the trickster character from folklore.

Can you make Anansi *fool* the judges into giving him a high score?  (Higher than some of the earlier argubots, while actually being worse at the task?)  **Any sneaky way of constructing Anansi's responses is fair game.**  The goal is to do well under automated evaluation on a held-out test set.  That is, Anansi should continue to score highly when talking to a character who is not in `evaluate.dev_chars` = {Bob, Cara, Darius, Eve, TrollFace}, when judged both by the character he is talking to and by Judge Wise.

To do well at this, figure out what the judges "want" -- what they might reward or respond positively to -- and how to give it to them.  This might be done by pure prompt engineering, or with additional computation (perhaps making use of additional LLM calls or other resources).  Again, explain what you did, and discuss how it worked out on the dev set.  Feel free to mention other ideas you had, too.